In [12]:
!pip install transformers

In [13]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import shutil
import sys

In [14]:
train_df = pd.read_csv("trainpreprocessed.csv")
train_df.head()

,Unnamed: 0,id,CONTEXT,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,Consumer protection,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,0,4772,consent order matter solium financial services...,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,4774,alberta securities commission investors top in...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4775,exempt market dealer settlement alberta securi...,0,0,0,1,0,1,0,...,0,0,0,0,0,0,1,1,0,1
3,3,4776,canadian securities regulators announces consu...,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,4,4778,csa consultation paper consideration access eq...,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0


In [15]:
train_df.shape

(9859, 53)

In [16]:
train_df.columns

Index(['Unnamed: 0', 'id', 'CONTEXT', 'Accounting and Finance', 'Antitrust',
       'Banking', 'Broker Dealer', 'Commodities Trading',
       'Compliance Management', 'Consumer protection', 'Contract Provisions',
       'Corporate Communications', 'Corporate Governance', 'Definitions',
       'Delivery', 'Examinations', 'Exemptions', 'Fees and Charges',
       'Financial Accounting', 'Financial Crime', 'Forms', 'Fraud', 'IT Risk',
       'Information Filing', 'Insurance', 'Legal', 'Legal Proceedings',
       'Licensing', 'Licensure and certification', 'Liquidity Risk', 'Listing',
       'Market Abuse', 'Market Risk', 'Monetary and Economic Policy',
       'Money Services', 'Money-Laundering and Terrorist Financing',
       'Natural Disasters', 'Payments and Settlements', 'Powers and Duties',
       'Quotation', 'Records Maintenance', 'Regulatory Actions',
       'Regulatory Reporting', 'Required Disclosures', 'Research',
       'Risk Management', 'Securities Clearing', 'Securities Issu

In [17]:
target_list = ['Accounting and Finance', 'Antitrust',
    'Banking', 'Broker Dealer', 'Commodities Trading',
    'Compliance Management', 'Consumer protection', 'Contract Provisions',
    'Corporate Communications', 'Corporate Governance', 'Definitions',
    'Delivery', 'Examinations', 'Exemptions', 'Fees and Charges',
    'Financial Accounting', 'Financial Crime', 'Forms', 'Fraud', 'IT Risk',
    'Information Filing', 'Insurance', 'Legal', 'Legal Proceedings',
    'Licensing', 'Licensure and certification', 'Liquidity Risk', 'Listing',
    'Market Abuse', 'Market Risk', 'Monetary and Economic Policy',
    'Money Services', 'Money-Laundering and Terrorist Financing',
    'Natural Disasters', 'Payments and Settlements', 'Powers and Duties',
    'Quotation', 'Records Maintenance', 'Regulatory Actions',
    'Regulatory Reporting', 'Required Disclosures', 'Research',
    'Risk Management', 'Securities Clearing', 'Securities Issuing',
    'Securities Management', 'Securities Sales', 'Securities Settlement',
    'Trade Pricing', 'Trade Settlement']
len(target_list)

50

In [18]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-05

In [19]:
from transformers import BertTokenizer , BertModel

In [20]:
tokenizer  = BertTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

In [21]:
example_texr = "hi my name is ritwik and this is my bert model"
encodings = tokenizer.encode_plus(
    example_texr,
    add_special_tokens = True,
    max_length = MAX_LEN,
    padding = "max_length",
    truncation = True,
    return_attention_mask = True,
    return_tensors = "pt"
)

In [22]:
encodings

{'input_ids': tensor([[ 101, 5996, 1357,  505,  223, 5770,  190, 6681,  181,  212,  226,  223,
         1357,  219,  188,  190, 1955,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,

In [23]:
class customdataset(torch.utils.data.Dataset) :
    def __init__(self , df , tokenizer , max_len) :
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.title = self.df['CONTEXT']
        self.targets = self.df[target_list].values
        
        
    def  __len__ (self) :
        return len(self.title)
    
    
    def __getitem__(self,index) :
        title  = str(self.title[index])
        titile = " ".join(title.split())
        
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens = True,
            max_length = MAX_LEN,
            padding = "max_length",
            truncation = True,
            return_attention_mask = True,
            return_tensors = "pt"
        )    
        
        return {
            'input_ids' : inputs['input_ids'].flatten(),
            'attention_mask' : inputs['attention_mask'].flatten(),
            'token_type_ids' : inputs['token_type_ids'].flatten(),
            'targets' : torch.FloatTensor(self.targets[index])
            
        }

In [24]:
train_size = 0.8
train_df = train_df.sample(frac=train_size , random_state=200).reset_index(drop=True)
valid_df = train_df.drop(train_df.index).reset_index(drop=True)

In [25]:
train_dataset = customdataset(train_df , tokenizer , MAX_LEN)
valid_dataset = customdataset(valid_df , tokenizer , MAX_LEN)

In [26]:
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle = True,
    batch_size = TRAIN_BATCH_SIZE,
    num_workers = 0
)
valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    shuffle = False,
    batch_size = TRAIN_BATCH_SIZE,
    num_workers = 0
)

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [28]:
def loadchcekpoint (checkpoint_fpath , model , optimizer) :
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min  = checkpoint['valid_loss_min']
    return model , optimizer , checkpoint['epoch'] , valid_loss_min.item()

def save_ckp (state , is_best , checkpoint_path , best_model_path) :
    f_path = checkpoint_path
    torch.save(state,f_path)
    if is_best :
        best_fpath = best_model_path
        shutil.copyfile(f_path,best_fpath)
        

In [29]:
class BERTCLass(nn.Module) :
    def __init__(self):
        super(BERTCLass,self).__init__() 
        self.bert_model = BertModel.from_pretrained("nlpaueb/legal-bert-base-uncased", return_dict = True)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(768,50)
    
    def forward (self,input_ids , attention_mask , token_type_ids) :
        output = self.bert_model(input_ids , attention_mask , token_type_ids)
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)

model = BERTCLass()
model.to(torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu"))            

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTCLass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [30]:
def lossfunction (outputs , targets) :
    return nn.BCEWithLogitsLoss()(outputs,targets)
optimizer = torch.optim.Adam(params=model.parameters() , lr =LEARNING_RATE)

In [31]:
def trainmodel (
    n_epochs,
    train_loader,
    valid_loader,
    model,
    optimizer,
    checkpoint_path,
    best_model_path
) :
    valid_loss_min = np.Inf
    
    for epoch in range (1 , n_epochs +1) :
        train_loss = 0
        valid_loss = 0
        model.train()
        for index , batch in enumerate(train_loader) :
            input_ids = batch['input_ids'].to(device,dtype = torch.long) 
            attention_mask = batch['attention_mask'].to(device,dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device,dtype = torch.long)
            targets = batch['targets'].to(device,dtype = torch.float)
            output = model(input_ids , attention_mask , token_type_ids)
            optimizer.zero_grad()
            loss = lossfunction(output,targets)
            optimizer.zer0_grad()
            loss.backward()
            optimizer.step()
            train_loss = train_loss + (1/(index+1)(loss.item() - train_loss))
            
        model.eval()
        with torch.no_grad():
            for index , batch in enumerate(valid_loader) :
                input_ids = batch['input_ids'].to(device,dtype = torch.long) 
                attention_mask : batch['attention_mask'].to(device,dtype = torch.long)
                token_type_ids : batch['token_type_ids'].to(device,dtype = torch.long)
                targets = batch['targets'].to(device,dtype = torch.float)
                output = model(input_ids , attention_mask , token_type_ids)
                loss = lossfunction(output,targets)
                valid_loss = valid_loss + (1/(index+1)(loss.item() - valid_loss))
            
        checkpoint = {
            'epoch' : epoch +1,
            'valid_loss_min' : valid_loss,
            'state_dict' : model.state_dict(),
            'optimizer' : optimizer.state_dict()
        }
        
        save_ckp(checkpoint,False,checkpoint_path,best_model_path)
        
    return model    

In [32]:
trained_model = trainmodel(EPOCHS,train_data_loader,valid_data_loader,model,optimizer,"/curr_ckpt","/best.pt") 